In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
import re
import datetime
import pymysql

### 1. Data Scraping 

#### url 입력 (자료가 있는 정례브리핑 url 주소 입력)

In [41]:
url = urllib.request.urlopen(input()).read()
#soup = bs.BeautifulSoup(url, 'lxml')

http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=354322&contSeq=354322&board_id=&gubun=ALL


#### pandas를 이용한 table scraping

In [42]:
dfs = pd.read_html(url, encoding = "utf-8", header=0)

#### 들쑥날쑥한 Column 수를 맞춰주기 위한 추가 코드

In [43]:
if len(dfs[0].columns) == 9 :
    dfs[0].drop(columns=dfs[0].columns[0], inplace = True)
    
dfs[0]

,구분.1,총계,결과 양성,결과 양성.1,결과 양성.2,결과 양성.3,검사 중,결과 음성
0,구분,총계,확진자,격리해제,격리 중,사망,검사 중,결과 음성
1,4. 30.(목) 0시 기준,619881,10765,9059,1459,247,8634,600482
2,5. 1.(금) 0시 기준,623069,10774,9072,1454,248,8685,603610
3,변동,"(+)3,188",(+)9,(+)13,(-)5,(+)1,(+)51,"(+)3,128"


#### domestic_check table에 들어갈 내용 scraping

In [44]:
column = dfs[0].columns
date = dfs[0][column[0]][2]
regex = re.compile(r'\d.')
date_group = regex.findall(date)

for i in range(0, len(date_group)):
    date_group[i] = int(re.sub("[^\d]", "", date_group[i]))
    
report_day = datetime.datetime(2020,date_group[0],date_group[1])
report_day_str = report_day.strftime('%Y-%m-%d')

check = []

for i in range(1, 8):
    check.append(int(dfs[0][column[i]][2]))
check.append(int(dfs[0][column[2]][3].replace("(+)", "")\
                                          .replace("(-)", "-")\
                                          .replace(",", "")))
check.append(int(dfs[0][column[3]][3].replace("(+)", "")\
                                          .replace("(-)", "-")\
                                          .replace(",", "")))
check.insert(0, report_day_str)
check

['2020-05-01', 623069, 10774, 9072, 1454, 248, 8685, 603610, 9, 13]

#### domestic_sid table에 들어갈 내용 scraping

In [45]:
column = dfs[1].columns
sid = []

for i in range(0, 3):
    list_temp = []
    for j in range(1, len(column)):
        list_temp.append(int(dfs[1][column[j]][i]))
    if i == 0 : list_temp.insert(0, "under quarantine")
    elif i == 1 : list_temp.insert(0, "end quarantine")
    elif i == 2 : list_temp.insert(0, "death")
    list_temp.insert(0, report_day_str)
    sid.append(list_temp)
sid

[['2020-05-01',
  'under quarantine',
  1454,
  177,
  18,
  536,
  25,
  3,
  5,
  5,
  7,
  174,
  11,
  4,
  15,
  6,
  3,
  165,
  19,
  5,
  276],
 ['2020-05-01',
  'end quarantine',
  9072,
  455,
  116,
  6144,
  69,
  27,
  35,
  37,
  39,
  488,
  40,
  41,
  128,
  12,
  12,
  1149,
  98,
  8,
  174],
 ['2020-05-01',
  'death',
  248,
  2,
  3,
  172,
  0,
  0,
  0,
  1,
  0,
  16,
  2,
  0,
  0,
  0,
  0,
  52,
  0,
  0,
  0]]

#### domestic_group table에 들어갈 내용 scraping

In [46]:
location = ["Seoul", "Busan", "Daegu", "Incheon",
 "Gwangju", "Daejeon", "Ulsan", "Sejong",
 "Gyeonggi", "Gangwon", "Chungbuk", "Chungnam",
 "Jeonbuk", "Jeonnam", "Gyeongbuk", "Gyeongnam", "Jeju", "Quarantine"]
column = dfs[2].columns
group = []

for i in range(2, 20):
    list_temp = []
    for j in range(1, 12):
        list_temp.append(int(dfs[2][column[j]][i]))
    list_temp.insert(0, location[i-2])
    list_temp.insert(0, report_day_str)
    list_temp.pop(5)
    list_temp.pop(10)
    group.append(list_temp)
group

ValueError: invalid literal for int() with base 10: '(1.8%)'

### 2. PYTHON과 MYSQL 연동

In [34]:
conn = pymysql.connect(host = '192.168.0.2', user = 'vanillapapaya',
                      password = 'qhans7810!', db = 'covid',
                      charset = 'utf8')
curs = conn.cursor()

#### domestic_check에 column 추가

In [35]:
query = """

        INSERT INTO domestic_check(Date, Total, Positive, EndQuarantine, UnderQuarantine, Death,
                               UnderExamination, Negative, NewPositive, NewEndQuarantine)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        
"""

curs.execute(query, check)

1

#### domestic_sid에 column 추가

In [36]:
query = """

        INSERT INTO domestic_sid(Date, Type, Total, Seoul, Busan, Daegu, Incheon, Gwangju, Daejeon,
                               Ulsan, Sejong, Gyeonggi, Gangwon, Chungbuk, Chungnam, Jeonbuk, Jeonnam,
                               Gyeongbuk, Gyeongnam, Jeju, Quarantine)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

"""
curs.executemany(query, sid)

3

#### domestic_group에 column 추가

In [37]:
query = """

        INSERT INTO domestic_group(Date, Location, Total, ForeignInflow, GroupTotal, 
                                    Shincheonji, GroupCases, PersonsInContact, ForeignGroupInflow, Other, New)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

"""
curs.executemany(query, group)

18

#### 변경된 내용 확인

In [39]:
query = """

    SELECT *
        FROM domestic_check

"""

df = pd.read_sql(query, conn)
df

,index,Date,Positive,EndQuarantine,UnderQuarantine,Death,UnderExamination,Negative,Total,NewPositive,NewEndQuarantine
0,0.0,2020-01-21,1,0,1,0,3,7,11,0,0
1,1.0,2020-01-24,2,0,2,0,0,25,27,1,0
2,2.0,2020-01-26,3,0,3,0,1,47,51,1,0
3,3.0,2020-01-27,4,0,4,0,1,56,61,1,0
4,4.0,2020-01-30,6,0,6,0,41,199,246,2,0
...,...,...,...,...,...,...,...,...,...,...,...
90,NaN,2020-04-26,10728,8717,1769,242,8999,578558,598285,10,82
91,NaN,2020-04-27,10738,8764,1731,243,8895,582027,601660,10,47
92,NaN,2020-04-28,10752,8854,1654,244,9203,588559,608514,14,90
93,NaN,2020-04-29,10761,8922,1593,246,8307,595129,614197,9,68


#### QUERY 실행을 완료한 후 MYSQL에 최종 적용

In [40]:
conn.commit()